In [1]:
import pandas as pd
from flask import Flask, request, jsonify
import threading
from flask_cors import CORS
import requests
import glob

In [2]:
# Combinar los archivos CSV
file_list = glob.glob('conteos_por_estado*.csv')
dfs = [pd.read_csv(file) for file in file_list]
combined_df = pd.concat(dfs, ignore_index=True)
dfs = []

for file in file_list:
    df = pd.read_csv(file)
        
    # Homologar los nombres de los estados
    df['Ciudad de Mexico'] = df.get('CDMX', 0) + df.get('Ciudad de Mexico', 0) + df.get('Distrito', 0)
    df['Mexico'] = df.get('Estado de Mexico', 0) + df.get('Mexico', 0)

     # Seleccionar solo las columnas relevantes
    df = df[['mes_año', 'Ciudad de Mexico', 'Mexico']]

    dfs.append(df)

# Combinar todos los dataframes
combined_df = pd.concat(dfs, ignore_index=True)

# Homologar los nombres de los estados
""" combined_df['estado_homologado'] = combined_df['estado'].replace({
    'Estado de Mexico': 'Mexico',
    'CDMX': 'Ciudad de Mexico',
    'Ciudad de Mexico': 'Ciudad de Mexico',
    'Distrito': 'Ciudad de Mexico'
}) """

# Guardar el dataframe combinado y homologado
#combined_df.to_csv('mensajes_limpios.csv', index=False)


" combined_df['estado_homologado'] = combined_df['estado'].replace({\n    'Estado de Mexico': 'Mexico',\n    'CDMX': 'Ciudad de Mexico',\n    'Ciudad de Mexico': 'Ciudad de Mexico',\n    'Distrito': 'Ciudad de Mexico'\n}) "

In [3]:
homologacion_estados = {
    'estado de mexico': 'mexico',
    'méxico': 'mexico',
    'estado de méxico': 'mexico',
    'cdmx': 'ciudad de mexico',
    'distrito': 'ciudad de mexico',
    'distrito federal': 'ciudad de mexico',
    'ciudad de méxico': 'ciudad de mexico'
}

In [4]:
# Función para cargar y homologar estados
def cargar_y_homologar_archivos(file_list, homologacion_estados):
    dfs = []

    for file in file_list:
        df = pd.read_csv(file)
        # Normalizar los nombres de las columnas a minúsculas
        df.columns = [col.lower() for col in df.columns]
        
        # Homologar los nombres de los estados
        for original, homologado in homologacion_estados.items():
            if original in df.columns:
                if homologado in df.columns:
                    df[homologado] = df[homologado].fillna(0) + df[original].fillna(0)
                else:
                    df[homologado] = df[original].fillna(0)
                df.drop(columns=[original], inplace=True)

        dfs.append(df)

    # Combinar todos los dataframes
    combined_df = pd.concat(dfs, ignore_index=True)

    # Unir las columnas de estado en una sola columna 'estado' y sumar la cantidad de mensajes
    estado_cols = combined_df.columns.difference(['mes_año'])
    homologado = combined_df.melt(id_vars=['mes_año'], value_vars=estado_cols, var_name='estado', value_name='cantidad_mensajes')

    # Convertir la columna 'mes_año' a un formato consistente (año-mes-día) solo si el formato es incorrecto
    def convertir_fecha(fecha):
        try:
            # Intentar convertir el formato incorrecto primero
            if len(fecha) == 10 and fecha[2] == '/' and fecha[5] == '/':
                fecha_formateada = pd.to_datetime(fecha, format='%d/%m/%Y', errors='coerce')
            else:
                fecha_formateada = pd.to_datetime(fecha, errors='coerce')
            
            if pd.isnull(fecha_formateada):
                return fecha  # Retorna la fecha original si no puede ser convertida
            
            return fecha_formateada.strftime('%Y-%m-%d')
        except:
            return fecha

    homologado['mes_año'] = homologado['mes_año'].apply(convertir_fecha)

    # Agrupar por fecha y estado homologado
    result_df = homologado.groupby(['mes_año', 'estado'])['cantidad_mensajes'].sum().reset_index()

    return result_df

In [5]:

# Obtener el dataframe combinado y homologado
result_df = cargar_y_homologar_archivos(file_list, homologacion_estados)


In [6]:
df2019 = pd.read_csv("conteos_mensuales2019.csv")
df2019["estado"] = df2019["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2019 = df2019.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2019[["Año", "Mes"]] = df2019["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2019 = df2019.drop(columns=["year_month"])

In [7]:
df2020 = pd.read_csv("conteos_mensuales2020.csv")
df2020["estado"] = df2020["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2020 = df2020.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2020[["Año", "Mes"]] = df2020["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2020 = df2020.drop(columns=["year_month"])

In [8]:
df2021 = pd.read_csv("conteos_mensuales_2021.csv")
df2021["estado"] = df2021["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2021 = df2021.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2021[["Año", "Mes"]] = df2021["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2021 = df2021.drop(columns=["year_month"])

In [9]:
df2022 = pd.read_csv("conteos_mensuales_2022.csv")
df2022["estado"] = df2022["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2022 = df2022.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2022[["Año", "Mes"]] = df2022["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2022 = df2022.drop(columns=["year_month"])

In [10]:
app = Flask(__name__)
CORS(app)

In [11]:
datasets = {"2019": df2019, "2020": df2020, "2021": df2021, "2022": df2022}

In [12]:
def fetch_holidays(api_key, country, year):
    url = "https://calendarific.com/api/v2/holidays"
    params = {"api_key": api_key, "country": country, "year": year}

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()  # Obtener el contenido JSON de la respuesta
        holidays = data.get("response", {}).get("holidays", [])
        return holidays
    else:
        print(f"Error: {response.status_code}")
        return []


# Llave API, país y años de ejemplo
api_key = "5Sqyan7wZsyg4nMiQzHKqMSxWq7oGThI"
country = "MX"
years = [2019, 2020, 2021, 2022]

# Obtener los datos de las efemérides para cada año
holidays_by_year = {year: fetch_holidays(api_key, country, year) for year in years}

In [13]:
# Extraer la información relevante
def extract_holiday_info(holidays):
    extracted_info = []
    for holiday in holidays:
        year = holiday['date']['datetime']['year']
        month = holiday['date']['datetime']['month']
        day = holiday['date']['datetime']['day']
        name = holiday['name']
        locations = holiday['locations']
        extracted_info.append({
            'year': year,
            'month': month,
            'day': day,
            'name': name,
            'locations': locations
        })
    return extracted_info

# Extraer la información de las efemérides para cada año
extracted_holidays_by_year = {year: extract_holiday_info(holidays_by_year[year]) for year in holidays_by_year}

In [14]:
# Filtrar las efemérides por año y mes
def filter_holidays_by_year_and_month(extracted_holidays_by_year, year, month):
    if year not in extracted_holidays_by_year:
        return []

    return [
        holiday
        for holiday in extracted_holidays_by_year[year]
        if holiday["month"] == month
    ]

In [15]:
@app.route('/mensajes', methods=['POST'])
def get_mensajes():
    request_data = request.get_json()
    year = request_data.get("año")
    month = request_data.get('month')

    if not year or not month:
        return jsonify({'error': 'Año y mes son requeridos'}), 400

    # Formatear el mes y el año
    mes_año = f'{year}-{month.zfill(2)}-01' 

    print(mes_año)
    # Filtrar los datos por año y mes
    filtered_df = result_df[result_df['mes_año'] == mes_año]

    # Convertir el resultado a un diccionario
    result = filtered_df.to_dict(orient='records')

    return jsonify(result)


In [16]:
@app.route("/get_data_by_year", methods=["POST"])
def get_data_by_year():
    request_data = request.get_json()
    estado = request_data.get("estado").upper().replace(" ", "")
    año = request_data.get("año")

    if año not in datasets:
        return jsonify({"error": "Año no válido"}), 400

    df = datasets[año]
    result = df[df["estado"] == estado]

    if result.empty:
        return (
            jsonify(
                {"error": "No se encontraron datos para los parámetros proporcionados"}
            ),
            404,
        )

    result = result.to_dict(orient="records")
    return jsonify(result), 200

In [17]:
@app.route("/get_data", methods=["POST"])
def get_data():
    request_data = request.get_json()
    estado = request_data.get("estado").upper().replace(" ", "")
    mes = int(request_data.get("mes"))
    año = int(request_data.get("año"))

    if str(año) not in datasets:
        return jsonify({"error": "Año no válido"}), 400

    df = datasets[str(año)]
    result = df[(df["estado"] == estado) & (df["Mes"] == f"{mes:02}")]

    if result.empty:
        return (
            jsonify(
                {"error": "No se encontraron datos para los parámetros proporcionados"}
            ),
            404,
        )

    result_data = result.to_dict(orient="records")

    # Obtener las festividades para el año y mes solicitados
    holidays = filter_holidays_by_year_and_month(extracted_holidays_by_year, año, mes)

    response = {"data": result_data, "holidays": holidays}

    return jsonify(response), 200

In [18]:
@app.route('/get_holidays_by_year', methods=['POST'])
def get_holidays_by_year():
    request_data = request.get_json()
    año = int(request_data.get('año'))

    if año not in extracted_holidays_by_year:
        return jsonify({"error": "Año no válido"}), 400

    holidays = extracted_holidays_by_year[año]

    if not holidays:
        return jsonify({"error": "No se encontraron efemérides para el año proporcionado"}), 404

    return jsonify(holidays), 200

In [ ]:
def run_app():
    app.run(debug=False, use_reloader=False)


# Ejecutar el servidor Flask en un hilo separado
threading.Thread(target=run_app).start()

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Jun/2024 01:19:44] "POST /get_data_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 01:19:50] "OPTIONS /mensajes HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 01:19:50] "POST /mensajes HTTP/1.1" 200 -


2019-02-01


127.0.0.1 - - [22/Jun/2024 01:20:44] "OPTIONS /mensajes HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 01:20:44] "POST /mensajes HTTP/1.1" 200 -


2019-02-01


127.0.0.1 - - [22/Jun/2024 01:22:20] "OPTIONS /mensajes HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 01:22:20] "POST /mensajes HTTP/1.1" 200 -


...

2019-02-01


127.0.0.1 - - [22/Jun/2024 02:30:39] "OPTIONS /get_data_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:30:39] "POST /get_data_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:30:39] "OPTIONS /get_holidays_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:30:39] "POST /get_holidays_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:30:57] "OPTIONS /get_data_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:30:57] "POST /get_data_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:30:57] "OPTIONS /get_holidays_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:30:57] "POST /get_holidays_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:30:57] "OPTIONS /mensajes HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:30:57] "POST /mensajes HTTP/1.1" 200 -


2019-02-01


127.0.0.1 - - [22/Jun/2024 02:31:02] "OPTIONS /get_data_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:31:02] "POST /get_data_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:31:02] "OPTIONS /get_holidays_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:31:02] "POST /get_holidays_by_year HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:31:03] "OPTIONS /mensajes HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 02:31:03] "POST /mensajes HTTP/1.1" 200 -


2019-02-01


In [ ]:
import requests
import json

url = "http://127.0.0.1:5000/get_data_by_year"
headers = {"Content-Type": "application/json"}
data = {"estado": "Ciudad de México", "año": "2019"}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Jun/2024 20:29:01] "POST /get_data_by_year HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2024 20:29:01] "POST /get_data_by_year HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2024 20:29:01] "POST /get_data HTTP/1.1" 200 -
127.0.0.1 - - [21/Jun/2024 20:29:01] "POST /mensajes HTTP/1.1" 200 -


[{'Año': '2019', 'Mes': '01', 'conteo': 341, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '02', 'conteo': 397, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '03', 'conteo': 544, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '04', 'conteo': 400, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '05', 'conteo': 405, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '06', 'conteo': 501, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '07', 'conteo': 618, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '08', 'conteo': 892, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '09', 'conteo': 718, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '10', 'conteo': 771, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '11', 'conteo': 579, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '12', 'conteo': 761, 'estado': 'CIUDADDEMÉXICO'}]


 * Serving Flask app '__main__'


In [ ]:
import requests
import json

url = "http://127.0.0.1:5000/get_data_by_year"
headers = {"Content-Type": "application/json"}
data = {"estado": "Ciudad de México", "año": "2019"}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

[{'Año': '2019', 'Mes': '01', 'conteo': 341, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '02', 'conteo': 397, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '03', 'conteo': 544, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '04', 'conteo': 400, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '05', 'conteo': 405, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '06', 'conteo': 501, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '07', 'conteo': 618, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '08', 'conteo': 892, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '09', 'conteo': 718, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '10', 'conteo': 771, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '11', 'conteo': 579, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '12', 'conteo': 761, 'estado': 'CIUDADDEMÉXICO'}]


In [ ]:
import requests
import json

url = "http://127.0.0.1:5000/get_data"
headers = {"Content-Type": "application/json"}
data = {"estado": "Aguascalientes", "año": "2019", "mes": "4"}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

{'data': [{'Año': '2019', 'Mes': '04', 'conteo': 23328, 'estado': 'AGUASCALIENTES'}], 'holidays': [{'day': 14, 'locations': 'All', 'month': 4, 'name': 'Palm Sunday', 'year': 2019}, {'day': 18, 'locations': 'All', 'month': 4, 'name': 'Maundy Thursday', 'year': 2019}, {'day': 19, 'locations': 'All', 'month': 4, 'name': 'Good Friday', 'year': 2019}, {'day': 20, 'locations': 'All', 'month': 4, 'name': 'Holy Saturday', 'year': 2019}, {'day': 21, 'locations': 'All', 'month': 4, 'name': 'Easter Sunday', 'year': 2019}, {'day': 30, 'locations': 'All', 'month': 4, 'name': "Children's Day", 'year': 2019}]}


In [ ]:
import requests
import json

url = "http://127.0.0.1:5000/mensajes"
headers = {"Content-Type": "application/json"}
data = {"año": "2019", "month": "04"}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

2019-04-01


[{'cantidad_mensajes': 23328, 'estado': 'aguascalientes', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 54032, 'estado': 'baja california', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 19991, 'estado': 'baja california sur', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 16554, 'estado': 'campeche', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 30274, 'estado': 'chiapas', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 35648, 'estado': 'chihuahua', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 491402, 'estado': 'ciudad de mexico', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 46967, 'estado': 'coahuila', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 11025, 'estado': 'colima', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 18526, 'estado': 'durango', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 183267, 'estado': 'estado de mexico', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 72195, 'estado': 'guanajuato', 'mes_año': '2019-04-01'}, {'cantidad_mensajes': 27565, 'esta